<a href="https://colab.research.google.com/github/pmontman/TSForecasting/blob/master/experiments/forecastingdata_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for accessing Forecastingdata forecasting and feature extraction tools using python

in Forecastingdata, some forecasting and feature calculation tools are implemented in R, but we can access them *transparently* from python using the awesome rpy2 package.
This notebook shows an example for it.


It has two parts. Part 1 is the preparation of the environment so everything
runs directly on python. Part2 is an example of how to use the tools directly from python.

## Part 1) Preparation of the environment

We load the R extension for jupyter, which interacts nicely with python

In [1]:
#@title
%load_ext rpy2.ipython

Clone the repository, for reproducibility. No need to clone everytime if you have already have the folder!.

In [2]:
!git clone https://github.com/pmontman/TSForecasting

fatal: destination path 'TSForecasting' already exists and is not an empty directory.


In [17]:
%cd TSForecasting/
!git pull
%cd ..

/Users/jerryobex/proj/TSForecasting-wrapper/TSForecasting


There is no tracking information for the current branch.
Please specify which branch you want to merge with.
See git-pull(1) for details.

    git pull <remote> <branch>

If you wish to set tracking information for this branch you can do so with:

    git branch --set-upstream-to=<remote>/<branch> main

/Users/jerryobex/proj/TSForecasting-wrapper


We now prepare the R environment for processing: installing and loading the required packages

```sh
cat << EOM > ~/.R/Makevars                                    
BREWDIR:=$(shell brew --prefix)
CPPFLAGS=-I$(BREWDIR)/include
LDFLAGS=-L$(BREWDIR)/lib
EOM

 brew install r harfbuzz fribidi
 brew install libgit2
 brew install freetype libpng libtiff zlib libjpeg-turbo
```


In [3]:
%%R
install.packages("devtools")
library("devtools")
install.packages("Rcatch22")

--- Please select a CRAN mirror for use in this session ---


Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Austria [https]
 6: Belgium (Brussels) [https]
 7: Brazil (PR) [https]
 8: Brazil (RJ) [https]
 9: Brazil (SP 1) [https]
10: Brazil (SP 2) [https]
11: Bulgaria [https]
12: Canada (MB) [https]
13: Canada (ON) [https]
14: Chile (Santiago) [https]
15: China (Beijing 2) [https]
16: China (Beijing 3) [https]
17: China (Hefei) [https]
18: China (Hong Kong) [https]
19: China (Guangzhou) [https]
20: China (Jinan) [https]
21: China (Lanzhou) [https]
22: China (Nanjing) [https]
23: China (Shanghai 2) [https]
24: China (Shenzhen) [https]
25: Colombia (Cali) [https]
26: Costa Rica [https]
27: Czech Republic [https]
28: Denmark [https]
29: Ecuador (Cuenca) [https]
30: France (Lyon 1) [https]
31: France (Lyon 2) [https]
32: France (Marseille) [https]
33: France (Paris 1) [https]
34: Germany (Erlangen) [https]
35: Germany (Göttingen) [https]
36: German

* installing *source* package ‘devtools’ ...
** package ‘devtools’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (devtools)
* installing *source* package ‘Rcatch22’ ...
** package ‘Rcatch22’ successfully unpacked and MD5 sums checked
** using staged installation
** libs
using C compiler: ‘Apple clang version 15.0.0 (clang-1500.1.0.2.5)’
using C++ compiler: ‘Apple clang version 15.0.0 (clang-1500.1.0.2.5)’
using SDK: ‘MacOSX14.2.sdk’


clang -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG -I../inst/include -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c CO_AutoCorr.c -o CO_AutoCorr.o
clang -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG -I../inst/include -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c DN_HistogramMode_10.c -o DN_HistogramMode_10.o
clang -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG -I../inst/include -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c DN_HistogramMode_5.c -o DN_HistogramMode_5.o
clang -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG -I../inst/include -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c DN_Mean.c -o DN_Mean.o
clang -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG -I../inst/include -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/o

installing to /opt/homebrew/lib/R/4.3/site-library/00LOCK-Rcatch22/00new/Rcatch22/libs
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (Rcatch22)


Installing package into ‘/opt/homebrew/lib/R/4.3/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/devtools_2.4.5.tar.gz'
Content type 'application/x-gzip' length 374718 bytes (365 KB)
downloaded 365 KB


The downloaded source packages are in
	‘/private/var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/RtmpiCtPkE/downloaded_packages’
Loading required package: usethis
Installing package into ‘/opt/homebrew/lib/R/4.3/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/Rcatch22_0.2.1.tar.gz'
Content type 'application/x-gzip' length 191609 bytes (187 KB)
downloaded 187 KB


The downloaded source packages are in
	‘/private/var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/RtmpiCtPkE/downloaded_packages’


In [4]:
%%R
install.packages("tidyverse")
install.packages("tsibble")
install.packages("forecast")
install.packages("tsfeatures")
install.packages("smooth")

* installing *source* package ‘tidyverse’ ...
** package ‘tidyverse’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (tidyverse)
* installing *source* package ‘tsibble’ ...
** package ‘tsibble’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed pack

clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArmadillo/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c calcBATS.cpp -o calcBATS.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArmadillo/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c calcTBATS.cpp -o calcTBATS.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArmadillo/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c etsTargetFunction.cpp -o etsTargetFunction.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArm

installing to /opt/homebrew/lib/R/4.3/site-library/00LOCK-forecast/00new/forecast/libs
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (forecast)
* installing *source* package ‘tsfeatures’ ...
** package ‘tsfeatures’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed packag

clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArmadillo/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c RcppExports.cpp -o RcppExports.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArmadillo/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c adamGeneral.cpp -o adamGeneral.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArmadillo/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c adamRefitter.cpp -o adamRefitter.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArm

installing to /opt/homebrew/lib/R/4.3/site-library/00LOCK-smooth/00new/smooth/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (smooth)


Installing package into ‘/opt/homebrew/lib/R/4.3/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/tidyverse_2.0.0.tar.gz'
Content type 'application/x-gzip' length 704618 bytes (688 KB)
downloaded 688 KB


The downloaded source packages are in
	‘/private/var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/RtmpiCtPkE/downloaded_packages’
Installing package into ‘/opt/homebrew/lib/R/4.3/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/tsibble_1.1.4.tar.gz'
Content type 'application/x-gzip' length 1552269 bytes (1.5 MB)
downloaded 1.5 MB


The downloaded source packages are in
	‘/private/var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/RtmpiCtPkE/downloaded_packages’
Installing package into ‘/opt/homebrew/lib/R/4.3/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/forecast_8.21.1.tar.gz'
Content type 'application/x-gzip' length 591793 bytes (577 KB)
downloaded 577 KB



Install glmnet package for linear models

In [20]:
%%R
install.packages('glmnet')

* installing *source* package ‘glmnet’ ...
** package ‘glmnet’ successfully unpacked and MD5 sums checked
** using staged installation
** libs
using C compiler: ‘Apple clang version 15.0.0 (clang-1500.1.0.2.5)’
using Fortran compiler: ‘GNU Fortran (Homebrew GCC 13.2.0) 13.2.0’
using C++ compiler: ‘Apple clang version 15.0.0 (clang-1500.1.0.2.5)’
using C++17
using SDK: ‘MacOSX14.2.sdk’


clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include' -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include   -Iglmnetpp/include -Iglmnetpp/src -Iglmnetpp/test -DEIGEN_PERMANENTLY_DISABLE_STUPID_WARNINGS -fPIC  -g -O2  -c RcppExports.cpp -o RcppExports.o
/opt/homebrew/opt/gcc/bin/gfortran  -fPIC  -g -O2  -c coxnet5dpclean.f -o coxnet5dpclean.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include' -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include   -Iglmnetpp/include -Iglmnetpp/src -Iglmnetpp/test -DEIGEN_PERMANENTLY_DISABLE_STUPID_WARNINGS -fPIC  -g -O2  -c elnet_exp.cpp -o elnet_exp.o


In file included from elnet_exp.cpp:2:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/RcppEigen.h:25:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/RcppEigenForward.h:28:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/Dense:1:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/Core:367:
/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/src/Core/util/Meta.h:320:25: warning: 'result_of<Eigen::internal::scalar_sum_op<double> (const double &, const double &)>' is deprecated [-Wdeprecated-declarations]
  typedef typename std::result_of<T>::type type1;
                        ^
/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/src/Core/CwiseBinaryOp.h:33:20: note: in instantiation of template class 'Eigen::internal::result_of<Eigen::internal::scalar_sum_op<double> (const double &, const double &)>' requested here
  typedef typename result_of<


clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include' -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include   -Iglmnetpp/include -Iglmnetpp/src -Iglmnetpp/test -DEIGEN_PERMANENTLY_DISABLE_STUPID_WARNINGS -fPIC  -g -O2  -c glmnet_init.cpp -o glmnet_init.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include' -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include   -Iglmnetpp/include -Iglmnetpp/src -Iglmnetpp/test -DEIGEN_PERMANENTLY_DISABLE_STUPID_WARNINGS -fPIC  -g -O2  -c internal.cpp -o internal.o
clang -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include' -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c pb.c -o pb.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4

In file included from wls_exp.cpp:2:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/RcppEigen.h:25:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/RcppEigenForward.h:28:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/Dense:1:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/Core:367:
/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/src/Core/util/Meta.h:320:25: warning: 'result_of<Eigen::internal::scalar_sum_op<double> (const double &, const double &)>' is deprecated [-Wdeprecated-declarations]
  typedef typename std::result_of<T>::type type1;
                        ^
/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/src/Core/CwiseBinaryOp.h:33:20: note: in instantiation of template class 'Eigen::internal::result_of<Eigen::internal::scalar_sum_op<double> (const double &, const double &)>' requested here
  typedef typename result_of<
  

clang++ -std=gnu++17 -dynamiclib -Wl,-headerpad_max_install_names -undefined dynamic_lookup -L/opt/homebrew/Cellar/r/4.3.2/lib/R/lib -L/opt/homebrew/lib -o glmnet.so RcppExports.o coxnet5dpclean.o elnet_exp.o glmnet_init.o internal.o pb.o pb_exp.o wls_exp.o -L/opt/homebrew/opt/gcc/lib/gcc/current/gcc/aarch64-apple-darwin23/13 -L/opt/homebrew/opt/gcc/lib/gcc/current/gcc -L/opt/homebrew/opt/gcc/lib/gcc/current -lgfortran -lemutls_w -lquadmath -L/opt/homebrew/Cellar/r/4.3.2/lib/R/lib -lR -lintl -Wl,-framework -Wl,CoreFoundation


** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (glmnet)


Installing package into ‘/opt/homebrew/lib/R/4.3/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/glmnet_4.1-8.tar.gz'
Content type 'application/x-gzip' length 2439515 bytes (2.3 MB)
downloaded 2.3 MB


The downloaded source packages are in
	‘/private/var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/Rtmp9B1k1m/downloaded_packages’


In [21]:
! ls TSForecasting/experiments

deep_learning_experiments.py fixed_horizon_functions.R
feature_experiments.R        forecastingdata_python.ipynb
feature_functions.R          rolling_origin.R
fixed_horizon.R


In [22]:
%%R
BASE_DIR <- "TSForecasting"
source("TSForecasting/experiments/feature_functions.R")
source("TSForecasting/experiments/fixed_horizon_functions.R")

The fixed horizon we need to try line by line to skip errors

In [23]:
%%R
#source("TSForecasting/experiments/fixed_horizon_functions.R")

## Part 2) Using tools directly on Python

Load the function into python environment through rpy2 so
it can be later accessed directly from python code

In [24]:
import rpy2.robjects as robjects
calculate_features = robjects.r['calculate_features']
do_fixed_horizon_local_forecasting = robjects.r['do_fixed_horizon_local_forecasting']
do_fixed_horizon_global_forecasting = robjects.r['do_fixed_horizon_global_forecasting']


we can now call `calculate_features` directly from python, here we have two examples using both the 'tsfeatures' and the 'catch22' features.
We will first remove the files just in case

Compute the features, we capture the stdout output to remove the verbosiness  of the progress report

In [26]:
%%capture cap --no-stderr
calculate_features("bitcoin_without_missing", "bitcoin_dataset_without_missing_values.tsf", "series_name", "start_timestamp", "tsfeatures")
calculate_features("bitcoin_without_missing", "bitcoin_dataset_without_missing_values.tsf", "series_name", "start_timestamp", "catch22")

The functions write the outputs to .csv files in the project directory
structure. We can quickly check if the experiments runs OK, we should see some features in the following files.

In [28]:
!echo "head of the tsfeatures features"
!ls -r TSForecasting/results/tsfeatures
!head -5 TSForecasting/results/tsfeatures/bitcoin_without_missing_features.csv
!echo "head of the catch22 features"
!ls -r TSForecasting/results/catch22_features
!head -5 TSForecasting/results/catch22_features/bitcoin_without_missing_features.csv

head of the tsfeatures features


sample_features.csv                  bitcoin_without_missing_features.csv
mean,var,max_kl_shift,time_kl_shift,max_level_shift,time_level_shift,max_var_shift,time_var_shift,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,ARCH.LM,crossing_points,entropy,flat_spots,alpha,beta,hurst,lumpiness,nonlinearity,x_pacf5,diff1x_pacf5,diff2x_pacf5,seas_pacf,stability,unitroot_kpss,unitroot_pp,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,peak,trough
3444499785151.05,3.93319052332885e+25,32.1557892841363,4569,0.859134378044087,4564,0.205497692454386,4561,0.999422064119912,9.92265154389631,0.243180998871893,0.0591389663416056,-0.339045902166888,0.14308972580683,0.995015161427049,0.997017749972443,3,0.122702786756397,3310,0.999752651409017,0.173992313970361,0.999954177318763,2.76285578022944e-05,0.067710254428927,1.01094602055946,0.063430096420893,0.320162765099821,-0.000364518933209693,0.999533535085673,25.3504716217215,-0.49734326051

Compute the local forecasts.

In [30]:
do_fixed_horizon_local_forecasting("bitcoin_without_missing", "ses", "bitcoin_dataset_without_missing_values.tsf", "series_name", "start_timestamp", 8)

[1] "Started loading bitcoin_without_missing"
[1] "started Forecasting"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] "Finished Forecasting"
Time difference of 0.3150501 secs
[1] "Mean SMAPE: 0.172689338373928"
[1] "Median SMAPE: 0.123138232829033"
[1] "Mean mSMAPE: 0.169545276372418"
[1] "Median mSMAPE: 0.122846077293209"
[1] "Mean MASE: 2.68851824080359"
[1] "Median MASE: 1.98823634639017"
[1] "Mean MAE: 319097329902355904"
[1] "Median MAE: 23914.3455388783"
[1] "Mean RMSE: 410102358991677056"
[1] "Median RMSE: 27805.9580413473"


<rpy2.rinterface_lib.sexp.NULLType object at 0x1044e2740> [RTYPES.NILSXP]

Try the global model forecasts.

In [31]:
do_fixed_horizon_global_forecasting("bitcoin_without_missing", 25, "bitcoin_dataset_without_missing_values.tsf", "pooled_regression", "series_name", "start_timestamp", 8)

[1] "Started loading bitcoin_without_missing"
[1] "started Forecasting"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] "Finished Forecasting"
Time difference of 0.5600109 secs
[1] "Mean SMAPE: 0.168943838542601"
[1] "Median SMAPE: 0.0846614286653965"
[1] "Mean mSMAPE: 0.165690860754268"
[1] "Median mSMAPE: 0.0844573447292264"
[1] "Mean MASE: 2.12338885164074"
[1] "Median MASE: 1.51821728104273"
[1] "Mean MAE: 468970463278232320"
[1] "Median MAE: 20952.7029991931"
[1] "Mean RMSE: 527537523615092224"
[1] "Median RMSE: 26067.0747261805"


<rpy2.rinterface_lib.sexp.NULLType object at 0x1044e2740> [RTYPES.NILSXP]